In [3]:
import numpy as np
import random
import cProfile

random.seed = 7

In [4]:
states = []
for i in range(10):
    for j in range(2):
        for k in range(10):
            states.append((i, j, k))

In [24]:
pi = {}
for state in states:
    pi[state] = {
        0 : 0,
        1 : 1
    }
for h in range(10):
    for a in range(2):
        for d in range(10):
            if h < 5:
                pi[(h, a, d)][1] = 1
                pi[(h, a, d)][0] = 0
                
            if d == 9 or d == 0:
                pi[(h, a, d)][1] = 0
                pi[(h, a, d)][0] = 1
                
            if (d < 6 and d > 0) and h < 6:
                pi[(h, a, d)][1] = 1
                pi[(h, a, d)][0] = 0
            
            if a == 1:
                pi[(h, a, d)][1] = 1
                pi[(h, a, d)][0] = 0
                
            if h > 7:
                pi[(h, a, d)][1] = 0
                pi[(h, a, d)][0] = 1
                

In [6]:
for h in range(10):
    for a in range(2):
        for d in range(10):
            if h > 6:
                pi[(h, a, d)][1] = 0
                pi[(h, a, d)][0] = 1
            else:
                pi[(h, a, d)][1] = 1
                pi[(h, a, d)][0] = 0

# Value Function
3 dimensional array, where first index is \[cardsum(12 to 21) - 12\] (_for indexing_); second index is 0 if no usable ace and 1 if any; third index is dealer card \[(1 to 10)-1\](_for indexing_).

In [7]:
V = np.zeros((10, 2, 10))
visit_count = np.zeros((10, 2, 10))

In [8]:
visit_count

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.,

# BlackJack Monte-Carlo Strategy

In [9]:
def pi_20(hand, dealer_card):
    player_sum = sum(hand)
    
    if 1 in hand and player_sum < 12:
            player_sum += 10
    
    if player_sum < 20:
        return 'hit'
    else:
        return 'stick'

In [10]:
def update_values(states_visited, r, V, visit_count):
    for h, a, d in states_visited:
        V[h][a][d] = V[h][a][d] + (1/visit_count[h][a][d])*(r - V[h][a][d])

# One Hand Blackjack Simulator

In [14]:
def blackjack_hand_result(V, visit_count):
    #Deck of cards, 4 types of 10
    
    deck=np.array([1,2,3,4,5,6,7,8,9,10,10,10,10])
    
    #Creating a random hand if none is input
    
    player_hand=[random.choice(deck), random.choice(deck)]
        
    dealer_card=random.choice(deck)
    
    # keeps track of all states that the hand has been in during one play
    states_visited = []
    
    player_sum = sum(player_hand)
    
    # an ace that can be used as an 11 i.e. the sum of the cards is less than 12
    usable_ace = 0
    if 1 in player_hand:
        usable_ace = 1
        player_sum += 10
    
    # update states
    if player_sum > 11:
        state = (player_sum-12, usable_ace, dealer_card-1)
        states_visited.append(state)
        visit_count[player_sum-12][usable_ace][dealer_card-1] += 1
        
    # keeps track of dealers hand
    dealer_cards=[dealer_card]
    

    # Seeing if player hit blackjack
    if player_sum == 21:

        #Seeing if casino also hit blackjack, in which case tie

        dealer_cards.append(random.choice(deck))
        if 1 in (dealer_cards) and sum(dealer_cards)==11:
            update_values(states_visited, 0, V, visit_count)
            return 0
        
        update_values(states_visited, 1, V, visit_count)
        return 1
    

    else:
    # Seeing how often it says to 'hit'
        hit = 1
        stay = 0
        while player_sum < 12 or pi[state][hit] == 1:
            #Adding one card for every hit
            new_card = random.choice(deck)
            player_hand.append(new_card)
            player_sum = sum(player_hand)
            
            usable_ace = 0
            
            if 1 in player_hand and player_sum < 12:
                usable_ace = 1
                player_sum += 10
            
            state = (player_sum-12, usable_ace, dealer_card-1)
            
            #Player loses bet if hand goes above 21
            if player_sum > 21:
                update_values(states_visited, -1, V, visit_count)
                return -1
            
            if player_sum > 11:
                states_visited.append(state)
                visit_count[player_sum-12][usable_ace][dealer_card-1] += 1
            
    while True:
        #Plays out the blackjack hand from dealer's side

        #Give dealer extra card if loop hasn't broken
        dealer_cards.append(random.choice(deck))


        #Keep track of sum of dealer's cards
        dealer_score= sum(dealer_cards)

        #Keep track of soft score if dealer has an ace

        soft_score= dealer_score
        if dealer_score<=11 and 1 in dealer_cards:
            soft_score+=10

        #If dealer gets blackjack you lose even if you have 21
        if len(dealer_cards)==2 and soft_score==21:
            update_values(states_visited, -1, V, visit_count)
            return -1
        
        #Dealer stays on all 17s
        if soft_score>=17:

            #If dealer bust, player wins bet
            if soft_score>21:
                update_values(states_visited, 1, V, visit_count)
                return 1

            #If player has more than dealer, player wins bet
            if player_sum>soft_score:
                update_values(states_visited, 1, V, visit_count)
                return 1

            #Tie means no money changes hands
            if player_sum==soft_score:
                update_values(states_visited, 0, V, visit_count)
                return 0

            #If player has lower, player loses bet
            if player_sum<soft_score:
                update_values(states_visited, -1, V, visit_count)
                return -1


# Multiple Hand Sim, Test to see house edge in Blackjack

In [12]:
def blackjack_sim(n_hands, V, visit_count):
    for i in range(n_hands):
        blackjack_hand_result(V, visit_count)
     

In [25]:
V = np.zeros((10, 2, 10))
visit_count = np.zeros((10, 2, 10))
cProfile.run("blackjack_sim(100_000, V, visit_count)")

         3686173 function calls in 2.756 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.428    0.000    0.428    0.000 <ipython-input-10-d6350201368c>:1(update_values)
        1    0.062    0.062    2.756    2.756 <ipython-input-12-f3b2d7b599ea>:1(blackjack_sim)
   100000    1.046    0.000    2.694    0.000 <ipython-input-14-317bdeb34f11>:1(blackjack_hand_result)
        1    0.000    0.000    2.756    2.756 <string>:1(<module>)
   505042    0.328    0.000    0.473    0.000 random.py:224(_randbelow)
   505042    0.321    0.000    0.841    0.000 random.py:256(choice)
        1    0.000    0.000    2.756    2.756 {built-in method builtins.exec}
   604104    0.057    0.000    0.057    0.000 {built-in method builtins.len}
   301122    0.146    0.000    0.146    0.000 {built-in method builtins.sum}
   100000    0.180    0.000    0.180    0.000 {built-in method numpy.core.multiarray.array}
   344088    0.042    0.

In [16]:
import plotly.graph_objects as go

y_values = []
x_values = []

for i in range(12, 22):
    for k in range(12, 22):
        x_values.append(i)

for i in range(10):
    for k in range(1, 11):
        y_values.append(k)



## Graph for hands without a usable ace

In [27]:
without_ace = np.array([[0.]*10]*10)
for i in range(len(V)):
    for j in range(len(V[0][0])):
        without_ace[i][j] = V[i][0][j]
        
z_values = np.array(without_ace).flatten()

without_ace_fig = go.Figure(data=[go.Mesh3d(x=np.array(x_values),
                   y=(np.array(y_values)),
                   z=(np.array(z_values)),
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])

without_ace_fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=10, range=[12, 21],),
                     yaxis = dict(nticks=10, range=[1, 10],),
                     zaxis = dict(nticks=5, range=[-1.5, 1.5],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

without_ace_fig.show()

In [26]:
V

array([[[-0.79147982, -0.52421959, -0.50166482, -0.52742616,
         -0.53169014, -0.52899408, -0.54416576, -0.44370123,
         -0.54928018, -0.55441887],
        [-0.5       , -0.39285714, -0.27272727, -0.41666667,
         -0.21052632, -0.28888889, -0.16923077, -0.39583333,
         -0.21568627, -0.25      ]],

       [[-0.72332943, -0.57023933, -0.55237045, -0.56935818,
         -0.59934853, -0.53480663, -0.52164502, -0.56071806,
         -0.54440961, -0.57300756],
        [-0.41414141, -0.20879121, -0.375     , -0.20454545,
         -0.34782609, -0.25      , -0.25714286, -0.16494845,
         -0.33333333, -0.36944444]],

       [[-0.72665148, -0.58061224, -0.55223881, -0.58283433,
         -0.65109034, -0.60460022, -0.58118812, -0.58057631,
         -0.64212679, -0.58887545],
        [-0.63265306, -0.21153846, -0.17171717, -0.42268041,
         -0.44791667, -0.10714286, -0.35964912, -0.26136364,
         -0.46      , -0.27317073]],

       [[-0.79217604, -0.65177665, -0.61431412

## Graph for Hands with a usable ace

In [28]:
with_ace = np.array([[0.]*10]*10)
for i in range(len(V)):
    for j in range(len(V[0][0])):
        with_ace[i][j] = V[i][1][j]
        
z_values = np.array(with_ace).flatten()

with_ace_fig = go.Figure(data=[go.Mesh3d(x=np.array(x_values),
                   y=(np.array(y_values)),
                   z=(np.array(z_values)),
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])

with_ace_fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=10, range=[12, 21],),
                     yaxis = dict(nticks=10, range=[1, 10],),
                     zaxis = dict(nticks=5, range=[-1.5, 1.5],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

with_ace_fig.show()